In [1]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
from time import time
import itertools
%matplotlib inline

In [2]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)
        
class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = []
        for i in range(1, nrow + 1):
            for j in range(1, ncol + 1):
                moves.append((i,j))
        self.state = State(moves)     

    def gameOver(self, state):
        if state.utility != 0 or len(state.moves) == 0 :
            return True
        else:
            return False
        
    def result(self, move, state): 
        new_state = cp.deepcopy(state)
        new_state.board[move] = new_state.to_move                    
        new_state.moves.remove(move) 
        if state.to_move=='X':
            new_state.to_move = 'O'  
        else:
            new_state.to_move = 'X'  
    
        new_state.utility = self.compute_utility(move, state)
        return new_state
        
    def compute_utility(self, move, state):
        '''
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''
        new_state = cp.deepcopy(state)
        new_state.board[move] = new_state.to_move                    
        new_state.moves.remove(move) 
        if state.to_move=='X':
            new_state.to_move = 'O'  
        else:
            new_state.to_move = 'X' 
        moveResult = new_state
        
        win = False 
        draw = False
        
        #checks to see if any horizontal wins have been made
        for i in range(1, self.nrow + 1):
            if state.to_move == moveResult.board.get((i, 1)):
                winCounter = 0
                for j in range(1, self.ncol + 1):
                    if state.to_move == moveResult.board.get((i, j)):
                        winCounter += 1
                if winCounter == self.nwin:
                    win = True
       
        #vertical wins
        for i in range(1, self.ncol + 1):
            if state.to_move == moveResult.board.get((1, i)):
                winCounter = 0
                for j in range(1, self.nrow + 1):
                    if state.to_move == moveResult.board.get((j, i)):
                        winCounter += 1
                if winCounter == self.nwin:
                    win = True
                    
        winCounter = 0
        for i in range(1, min(self.nrow + 1, self.ncol + 1)):
            if state.to_move == moveResult.board.get((i, i)):
                winCounter += 1
            if winCounter == self.nwin:
                win = True
                
        winCounter = 0        
        for i in range(1, min(self.nrow + 1, self.ncol + 1)):
            if state.to_move == moveResult.board.get((i, self.ncol + 1 - i)):
                winCounter += 1
            if winCounter == self.nwin:
                win = True
            
        if draw == True: # nobody has gotten 3-in-a-row
            return 0
        
        if win == True:
            if state.to_move=='X':
                return 1 
            else:
                return -1
        
        else: 
            return 0
    
    def utility(self, state, player):
        '''
        Return the value to the given player; 1 for win, -1 for loss, 0 otherwise.
        '''
        if player == state.to_move :
            return -1*state.utility
        else:
            return state.utility
    
        
        
    def display(self):
        '''Display the current game state.'''
        for row in range(1, self.nrow+1):
            for col in range(1, self.ncol+1):
                print(self.state.board.get((row, col), '.'), end=' ')
            print()
            

                 
    def play_game(self, player1, player2):
        '''
        Play a game of tic-tac-toe!
        player1 and player2 are function names, either random_player
        or alphabeta_player
        '''

        gameOva = self.gameOver(self.state)
        player = self.state.to_move
        while (gameOva == False):
            player = self.state.to_move
            moveX = player1(self)
            newStateX = self.result(moveX, self.state)
            self.state = newStateX
            gameOva = self.gameOver(self.state)
                #self.display()
                #print("-----")
                
            if gameOva == True:
                break
            player = self.state.to_move
            moveO = player2(self)
            newStateO = self.result(moveO, self.state)
            self.state = newStateO
            gameOva = self.gameOver(self.state)
                #self.display()
                #print("-----")
        #print(player)
        winnerUtility = self.utility(self.state, player)
        #self.display()
        #print(winner)
        return winnerUtility

In [3]:
def random_player(thing):
    i = np.random.choice(len(thing.state.moves))
    move = thing.state.moves[i]
    return move

In [4]:
randomProb = np.zeros(1000)
for i in range(0, 1000):  
    Hi = TicTacToe()
    randomProb[i] = Hi.play_game(random_player, random_player)

wins = [i for i in randomProb if i == 1]
losses = [i for i in randomProb if i == -1]
draws = [i for i in randomProb if i == 0]
print("Wins:", len(wins))
print("Losses:", len(losses))
print("Draws:", len(draws))
print("Win percentage with two random players:", len(wins)/len(randomProb))

Wins: 584
Losses: 286
Draws: 130
Win percentage with two random players: 0.584


In [5]:
randomProb = np.zeros(1000)
for i in range(0, 1000):  
    Hi = TicTacToe(4,4,4)
    randomProb[i] = Hi.play_game(random_player, random_player)
    
wins = [i for i in randomProb if i == 1]
losses = [i for i in randomProb if i == -1]
draws = [i for i in randomProb if i == 0]
print("Wins:", len(wins))
print("Losses:", len(losses))
print("Draws:", len(draws))
print("Win percentage of two random players on a 4x4 tic-tac-toe board:", len(wins)/len(randomProb))

Wins: 329
Losses: 247
Draws: 424
Win percentage of two random players on a 4x4 tic-tac-toe board: 0.329


In [6]:
def alphabeta_player(tttThing):
    state = tttThing.state
    if state.to_move == 'X':
        
        value = -np.inf
        alpha = -np.inf
        beta = np.inf
        bestMove = None

        for move in state.moves:
            nextState = tttThing.result(move, state)
            nextValue = min_value(tttThing, nextState, alpha, beta)
            if value < nextValue:
                value = nextValue
                bestMove = move
            if value > alpha:
                alpha = value
    else:
        value = np.inf
        alpha = -np.inf
        beta = np.inf
        bestMove = None
        
        for move in state.moves:
            nextState = tttThing.result(move, state)
            nextValue = max_value(tttThing, nextState, alpha, beta)
            if value > nextValue:
                value = nextValue
                bestMove = move
            if value < beta:
                beta = value
    return bestMove

def max_value(tttThing, state, alphaMax, betaMax):
    
    lastPlayer = 'X' if state.to_move == 'O' else 'X'
    if tttThing.gameOver(state) == True:
        return state.utility
        
    a = alphaMax
    value = -np.inf
    for move in state.moves:
        nextState = tttThing.result(move, state)
        nextValue = min_value(tttThing, nextState, a, betaMax)
        
        if value < nextValue:
            value = nextValue    
        if value > a:
            a = value
        
        if value >= betaMax:
            return value
    return value

def min_value(tttThing, state, alphaMin, betaMin):
    
    lastPlayer = 'X' if state.to_move == 'O' else 'X'
    if tttThing.gameOver(state) == True:
        return state.utility
        
    b = betaMin
    value = np.inf
    for move in state.moves:
        
        nextState = tttThing.result(move, state)
        nextValue = max_value(tttThing, nextState, alphaMin, b)
        
        if value > nextValue:
            value = nextValue
            
        if value < b:
            b = value
        
        if value <= alphaMin:
            return value
      
        
    return value


Test:

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw, and should always end up in the same terminal state.

In [7]:
randomProb = []
for i in range(0, 30):  
    Hi = TicTacToe()
    randomProb.append(Hi.play_game(alphabeta_player, random_player))
    #Hi.display()
    #print(i)
wins = [i for i in randomProb if i == 1]
losses = [i for i in randomProb if i == -1]
draws = [i for i in randomProb if i == 0]
print("Wins:", len(wins))
print("Losses:", len(losses))
print("Draws:", len(draws))
print("Lose percentage for player 1 if alphabeta player starts first:", len(losses)/len(randomProb))

Wins: 30
Losses: 0
Draws: 0
Lose percentage for player 1 if alphabeta player starts first: 0.0


In [8]:
randomProb = []
for i in range(0, 30):  
    Hi = TicTacToe()
    randomProb.append(Hi.play_game(random_player, alphabeta_player))
    #Hi.display()
    #print(i)
wins = [i for i in randomProb if i == 1]
losses = [i for i in randomProb if i == -1]
draws = [i for i in randomProb if i == 0]
print("Wins:", len(wins))
print("Losses:", len(losses))
print("Draws:", len(draws))
print("Win percentage for random player agains alphabeta player:", len(wins)/len(randomProb))

Wins: 0
Losses: 21
Draws: 9
Win percentage for random player agains alphabeta player: 0.0


In [9]:
randomProb = []
for i in range(0, 30):  
    Hi = TicTacToe()
    randomProb.append(Hi.play_game(alphabeta_player, alphabeta_player))
    #print(i)
    
wins = [i for i in randomProb if i == 1]
losses = [i for i in randomProb if i == -1]
draws = [i for i in randomProb if i == 0]
print("Wins:", len(wins))
print("Losses:", len(losses))
print("Draws:", len(draws))
print("Draw percentage with two alpha beta players:", len(draws)/len(randomProb))

Wins: 0
Losses: 0
Draws: 30
Draw percentage with two alpha beta players: 1.0
